In [43]:
import numpy as np
import sys

# Checks if pair is complementary and assigns the appropriate score as defined in the paper
def isPairScore(seq, i, j, probL, probR, probP):
  possibleBasePairs = set(['AU', 'UA', 'GC', 'CG', 'GU', 'UG']) # includes wobble pairs
  Ri = seq[i]
  Rj = seq[j]

  if Ri + Rj in possibleBasePairs:
    return probL[i] + probR[j]
  else:
    return probP[i] + probP[j]


# Initializes an n x n matrix
def initMatrix(Text : str) -> list[list[int]]:
  n = len(Text)

  M = np.empty([n, n])
  M[:] = np.NAN

  M[range(n), range(n)] = 0
  M[range(1, n), range(n - 1)] = 0

  return M

# Forward direction for Updated Nussinov algorithm
def Forward(probL, probR, probP, M, Text, min_loop_length = 0):
  n = len(Text)

  for k in range(1, n):
    for i in range(n - k):
      j = i + k

      if (j - i >= min_loop_length):
        down = M[i + 1][j]
        left = M[i][j - 1]
        diag = M[i + 1][j - 1] + isPairScore(Text, i, j, probL, probR, probP)
        other = max([M[i][t] + M[t + 1][j] for t in range(i, j)])

        M[i][j] = max(down, left, diag, other)
      else:
        M[i][j] = 0

  return M

# Back direction of Updated Nussinov algorithm, backtrack
def Back(M, Text, pairs, i, length, probL, probR, probP):
  j = length

  if i < j:
    if M[i][j] == M[i + 1][j]:
      Back(M, Text, pairs, i + 1, j, probL, probR, probP)
    elif M[i][j] == M[i][j - 1]:
      Back(M, Text, pairs, i, j - 1, probL, probR, probP)
    elif M[i][j] == M[i + 1][j - 1] + isPairScore(Text, i, j, probL, probR, probP):
      pairs.append((i, j))
      Back(M, Text, pairs, i + 1, j - 1, probL, probR, probP)
    else:
      for k in range(i + 1, j - 1):
        if M[i][j] == M[i, k] + M[k + 1][j]:
          Back(M, Text, pairs, i, k, probL, probR, probP)
          Back(M, Text, pairs, k + 1, j, probL, probR, probP)
          break


# Convert to dotBracket
def dotBracket(Text : str, pairs : list[tuple[int, int]]) -> str:
  dot = ["." for i in range(len(Text))]

  for s in pairs:
    dot[max(s)] = ")"
    dot[min(s)] = "("

  return "".join(dot)

# Run Updated Nussinov algorithm
def UpdatedNussinov(Text, probL, probR, probP):
  n = len(Text)

  initial_matrix = initMatrix(Text)
  forward_matrix = Forward(probL, probR, probP, initial_matrix, Text)

  pairs = []
  Back(forward_matrix, Text, pairs, 0, n - 1, probL, probR, probP)

  return dotBracket(Text, pairs)

In [44]:
from keras.layers import Conv2D, Dense, MaxPooling2D, Flatten
import keras

In [45]:
model = keras.models.load_model("82.h5")

In [46]:
import numpy as np
import math

def encode(seq):
    n = len(seq)
    l = [[0 for i in range(n)] for j in range(n)]
    for i in range(n):
        for j in range(n):
            if((seq[i] == 'A' and seq[j] == 'U') or (seq[j] == 'A' and seq[i] == 'U')):
                l[i][j] = 2
            elif((seq[i] == 'G' and seq[j] == 'C') or (seq[j] == 'G' and seq[i] == 'C')):
                l[i][j] = 3
            elif((seq[i] == 'G' and seq[j] == 'U') or (seq[j] == 'U' and seq[i] == 'G')):
                l[i][j] = 0.8
            else:
                l[i][j] = 0
    return l

def Gaussian(x):
    return math.exp(-0.5*(x*x))

def matrix_calc(data, sliding_param):
    mat = np.zeros([len(data)+sliding_param,len(data)])
    for i in range(len(data[0])):
        for j in range(len(data[0])):
            coefficient = 0
            for add in range(30):
                if i - add >= 0 and j + add <len(data):
                    score = data[i-add][j+add]
                    if score == 0:
                        break
                    else:
                        coefficient = coefficient + score * Gaussian(add)
                else:
                    break
            if coefficient > 0:
                for add in range(1,30):
                    if i + add < len(data) and j - add >= 0:
                        score = data[i+add][j-add]
                        if score == 0:
                            break
                        else:
                            coefficient = coefficient + score * Gaussian(add)
                    else:
                        break
            mat[[i+int(sliding_param/2)],[j]] = coefficient
    return mat

def slidingWindow(numpy_M, sliding_param, n):
    zeros = np.zeros((sliding_param, n), dtype=int)
    numpy_M = np.append(numpy_M, zeros, axis=0)
    sliding_mats = [numpy_M[i: i + sliding_param, :].tolist() for i in range(n)]
    return sliding_mats

def scale(im, nR, nC):
    nR0 = len(im) 
    nC0 = len(im[0]) 
    arr =  [[ im[int(nR0 * r / nR)][int(nC0 * c / nC)]  
             for c in range(nC)] for r in range(nR)]
    return np.array(arr)

In [47]:
seq = "GGGUCUGUAGCUCAGGUGGUUAGAGCGCACCCCUGAUAAGGGUGAGGUCGGUGGUUCGAGUCCUCCCAGACCCACCA"
windows = slidingWindow(matrix_calc(encode(seq), 19), 19, len(seq))
scaled = np.zeros(shape=(len(seq), 19, 120))
for i in range(len(windows)):
    scaled[i] = scale(windows[i], 19, 120)

In [48]:
print(scaled)

[[[0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  ...
  [0.         0.         0.         ... 0.         0.         2.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 3.         3.         0.        ]]

 [[0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  ...
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 3.         3.         0.        ]
  [3.         3.         3.         ... 0.         0.         0.        ]]

 [[0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.      

In [49]:
x = model.predict(scaled)

3/3 [==============================] - 0s 16ms/step


In [50]:
#print(x)
probL = []
probR = []
probP = []
for i in range(len(x)):
    probL.append(x[i][0])
    probR.append(x[i][1])
    probP.append(x[i][2])

print(UpdatedNussinov(seq, probL, probR, probP))

((((((((((((((().()()))))).(((((.(())()))))))()()((.(((()().)))))))))))))(())
